<a href="https://colab.research.google.com/github/Itsuki-Hamano123/auto_ml/blob/master/auto-keras/california_housing/defiend_ShallowNN_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip show tensorflow

Name: tensorflow
Version: 2.3.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: keras-preprocessing, numpy, google-pasta, opt-einsum, six, scipy, absl-py, tensorflow-estimator, tensorboard, astunparse, grpcio, gast, wrapt, protobuf, termcolor, wheel, h5py
Required-by: fancyimpute


In [3]:
%cd /content/drive/My Drive/機械学習練習/AutoML/auto-keras/california_housing
%ls

/content/drive/My Drive/機械学習練習/AutoML/auto-keras/california_housing
auto_keras_outputs/        datasets/                    fetch_data.ipynb
autokeras_regressor.ipynb  defiend_DNN_regressor.ipynb  xgb_regressor.ipynb


In [20]:
import cloudpickle, os, pprint

import pandas as pd
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score
from tensorflow.keras import Model, Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

## データ読み込み

In [5]:
F_COLS = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
TARGET_COL = 'MedHouseVal'

data_path = 'datasets/origin'
train_csv = 'train.csv'
val_csv = 'val.csv'
test_csv = 'test.csv'

train_df= pd.read_csv(os.path.join(data_path, train_csv), header=0)
val_df = pd.read_csv(os.path.join(data_path, val_csv), header=0)
test_df = pd.read_csv(os.path.join(data_path, test_csv), header=0)

print('train shape:{}'.format(train_df.shape))
print('validation shape:{}'.format(val_df.shape))
print('test shape:{}'.format(test_df.shape))

display(train_df.head())

train shape:(13209, 9)
validation shape:(3303, 9)
test shape:(4128, 9)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,4.4063,15.0,6.104430,1.003165,1237.0,3.914557,34.00,-117.50,1.285
1,3.7222,35.0,5.539326,1.030899,1152.0,3.235955,33.87,-117.96,1.875
2,1.1094,49.0,3.773585,1.144654,1448.0,4.553459,37.95,-121.28,0.525
3,3.3239,9.0,5.610955,1.092697,1691.0,2.375000,36.82,-119.85,0.913
4,5.9629,17.0,6.867133,1.097902,808.0,2.825175,38.58,-121.81,2.860


## Shallow Neural Networkモデルを構築

In [16]:
def model_fn(input_shape, hidden_size, output_size):
  '''モデルの定義'''
  model = Sequential([
    layers.Dense(hidden_size, activation='relu', input_shape=[None,input_shape]),
    layers.Dense(output_size)
  ])
  return model


model = model_fn(input_shape=len(F_COLS), hidden_size=64,
                 output_size=1)

model.compile(loss='mse',
              optimizer=Adam(learning_rate=0.001),
              metrics=['mae', 'mse'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, None, 64)          576       
_________________________________________________________________
dense_7 (Dense)              (None, None, 1)           65        
Total params: 641
Trainable params: 641
Non-trainable params: 0
_________________________________________________________________


In [17]:
%time
BATCH_SIZE = 256
EPOCHS = 50

es = EarlyStopping(patience=3)

model.fit(x=train_df[F_COLS].values, y=train_df[TARGET_COL].values,
          batch_size=BATCH_SIZE, epochs=EPOCHS,
          validation_data=(val_df[F_COLS].values, val_df[TARGET_COL].values),
          callbacks=[es]
          )

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.63 µs
Epoch 1/50
52/52 [==============================] - 0s 4ms/step - loss: 11076.4746 - mae: 57.1027 - mse: 11076.4746 - val_loss: 163.9289 - val_mae: 11.9892 - val_mse: 163.9289
Epoch 2/50
52/52 [==============================] - 0s 2ms/step - loss: 76.8025 - mae: 5.0169 - mse: 76.8025 - val_loss: 6.4963 - val_mae: 2.0304 - val_mse: 6.4963
Epoch 3/50
52/52 [==============================] - 0s 2ms/step - loss: 5.8402 - mae: 1.8837 - mse: 5.8402 - val_loss: 4.6593 - val_mae: 1.7428 - val_mse: 4.6593
Epoch 4/50
52/52 [==============================] - 0s 2ms/step - loss: 4.3347 - mae: 1.6474 - mse: 4.3347 - val_loss: 3.7885 - val_mae: 1.5731 - val_mse: 3.7885
Epoch 5/50
52/52 [==============================] - 0s 2ms/step - loss: 3.5481 - mae: 1.4863 - mse: 3.5481 - val_loss: 3.1116 - val_mae: 1.4196 - val_mse: 3.1116
Epoch 6/50
52/52 [==============================] - 0s 2ms/step - loss: 2.8613 - mae: 1.3308 - mse: 2.8613 - v

## 前処理したデータの読み込み
行った前処理
- 標準化

In [18]:
std_data_path = 'datasets/std'
std_train_csv = 'std_train.csv'
std_val_csv = 'std_val.csv'
std_test_csv = 'std_test.csv'

std_train_df= pd.read_csv(os.path.join(std_data_path, std_train_csv), header=0)
std_val_df = pd.read_csv(os.path.join(std_data_path, std_val_csv), header=0)
std_test_df = pd.read_csv(os.path.join(std_data_path, std_test_csv), header=0)

print('std_train shape:{}'.format(std_train_df.shape))
print('std_validation shape:{}'.format(std_val_df.shape))
print('std_test shape:{}'.format(std_test_df.shape))

display(std_train_df.head())

std_train shape:(13209, 9)
std_validation shape:(3303, 9)
std_test shape:(4128, 9)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,0.297052,-1.079924,0.251551,-0.185450,-0.170175,0.119535,-0.760080,1.026143,-0.667466
1,-0.062875,0.505159,0.040977,-0.132852,-0.242531,0.024947,-0.821013,0.796661,-0.152404
2,-1.437551,1.614717,-0.616988,0.082885,0.009438,0.208590,1.091352,-0.859600,-1.330936
3,-0.272433,-1.555449,0.067668,-0.015652,0.216291,-0.095059,0.561702,-0.146211,-0.992217
4,1.116028,-0.921416,0.535755,-0.005780,-0.535361,-0.032310,1.386643,-1.124003,0.707487


In [21]:
# 標準化器も読み込んでおく
def load_pkl(path):
  '''pklファイルの読み込み'''
  with open(path, 'rb') as f:
    obj = cloudpickle.load(f)
  return obj


pkl_name = 'std_scaler.pkl'
std_scaler = load_pkl(os.path.join(std_data_path, pkl_name))
std_scaler

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


StandardScaler(copy=True, with_mean=True, with_std=True)

## 前処理済みデータに対してモデルを構築

In [23]:
std_model = model_fn(input_shape=len(F_COLS), hidden_size=64,
                     output_size=1)

std_model.compile(loss='mse',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['mae', 'mse'])

std_model.fit(x=std_train_df[F_COLS].values, y=std_train_df[TARGET_COL].values,
              batch_size=BATCH_SIZE, epochs=EPOCHS,
              validation_data=(std_val_df[F_COLS].values, std_val_df[TARGET_COL].values),
              callbacks=[es]
              )

Epoch 1/50
52/52 [==============================] - 0s 4ms/step - loss: 0.8074 - mae: 0.6885 - mse: 0.8074 - val_loss: 0.5884 - val_mae: 0.5945 - val_mse: 0.5884
Epoch 2/50
52/52 [==============================] - 0s 2ms/step - loss: 0.4891 - mae: 0.5205 - mse: 0.4891 - val_loss: 0.4614 - val_mae: 0.4996 - val_mse: 0.4614
Epoch 3/50
52/52 [==============================] - 0s 2ms/step - loss: 0.4040 - mae: 0.4642 - mse: 0.4040 - val_loss: 0.4179 - val_mae: 0.4687 - val_mse: 0.4179
Epoch 4/50
52/52 [==============================] - 0s 2ms/step - loss: 0.3674 - mae: 0.4422 - mse: 0.3674 - val_loss: 0.3873 - val_mae: 0.4451 - val_mse: 0.3873
Epoch 5/50
52/52 [==============================] - 0s 2ms/step - loss: 0.3466 - mae: 0.4283 - mse: 0.3466 - val_loss: 0.3665 - val_mae: 0.4343 - val_mse: 0.3665
Epoch 6/50
52/52 [==============================] - 0s 2ms/step - loss: 0.3301 - mae: 0.4166 - mse: 0.3301 - val_loss: 0.3508 - val_mae: 0.4231 - val_mse: 0.3508
Epoch 7/50
52/52 [==========

## モデルの評価

### 評価指標の定義

In [26]:
SK_METRICS=[r2_score, explained_variance_score, mean_squared_error]

def _calc_score(y_true, y_pred, sklearn_metric):
  '''sklearnの評価指標のスコアを計算'''
  score = sklearn_metric(y_true=y_true, y_pred=y_pred)
  return score


def evaluete_sk_metrics(y_true, y_pred, sklearn_metrics):
  '''sklearnの各種、評価関数を使用'''
  result = {}
  for sk_metric in sklearn_metrics:
    result[sk_metric.__name__] = _calc_score(y_true=y_true, y_pred=y_pred, sklearn_metric=sk_metric)
  return result

### 推論結果の取得

In [27]:
def predict_multiple_times(model, inputs, input_names):
  '''何度か推論'''
  pred = {}
  for input, name in zip(inputs, input_names):
    pred[name] = model.predict(input)
  return pred

In [29]:
%time
reg_inputs = [train_df[F_COLS].values, val_df[F_COLS], test_df[F_COLS].values]
reg_input_names = [train_csv, val_csv, test_csv]

std_reg_inputs = [std_train_df[F_COLS].values, std_val_df[F_COLS].values, std_test_df[F_COLS].values]
std_input_names = [std_train_csv, std_val_csv, std_test_csv]


reg_preds = predict_multiple_times(model=model, inputs=reg_inputs, input_names=reg_input_names)
std_reg_preds = predict_multiple_times(model=std_model, inputs=std_reg_inputs, input_names=std_input_names)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs


### 評価の実行

In [33]:
model_name = 'ShallowNN'
std_model_name = 'std_ShallowNN'

eval_scores = {model_name: {}, std_model_name: {}}

for csv_name, y_true in zip([train_csv, val_csv, test_csv],
                            [train_df[TARGET_COL].values, val_df[TARGET_COL].values, test_df[TARGET_COL].values]):
  eval_scores[model_name][csv_name] = evaluete_sk_metrics(y_true=y_true,
                                                          y_pred=reg_preds[csv_name], sklearn_metrics=SK_METRICS)

for csv_name, y_true in zip([std_train_csv, std_val_csv, std_test_csv], 
                            [std_train_df[TARGET_COL].values, std_val_df[TARGET_COL].values, std_test_df[TARGET_COL].values]):
  eval_scores[std_model_name][csv_name] = evaluete_sk_metrics(y_true=y_true,
                                                              y_pred=std_reg_preds[csv_name], sklearn_metrics=SK_METRICS)

pprint.pprint(eval_scores)

{'ShallowNN': {'test.csv': {'explained_variance_score': 0.4349276989151253,
                            'mean_squared_error': 0.8056064330265855,
                            'r2_score': 0.4010434537196287},
               'train.csv': {'explained_variance_score': 0.4912248014795293,
                             'mean_squared_error': 0.7051086596845951,
                             'r2_score': 0.46263285548072286},
               'val.csv': {'explained_variance_score': 0.48019983175773084,
                           'mean_squared_error': 0.7613552447055286,
                           'r2_score': 0.45156132683995154}},
 'std_ShallowNN': {'std_test.csv': {'explained_variance_score': 0.746095717799165,
                                    'mean_squared_error': 0.26068363319909493,
                                    'r2_score': 0.7456855164473855},
                   'std_train.csv': {'explained_variance_score': 0.7600809596560849,
                                     'mean_squared_error': 

## モデルの予測値を標準化から復元し評価する場合

In [34]:
def inverse_scale_values(values, scaler):
  '''scalerを使って値の復元'''
  return scaler.inverse_transform(values)


inverse_scale_preds = {}
# 目的変数をモデルの予測値に置き換え
for df, csv_name, key in zip([std_train_df, std_val_df, std_test_df], [std_train_csv, std_val_csv, std_test_csv], 
                             [train_csv, val_csv, test_csv]):
  df[TARGET_COL] = std_reg_preds[csv_name]
  tmp_df = pd.DataFrame(inverse_scale_values(df.values, std_scaler), columns=F_COLS+[TARGET_COL])
  inverse_scale_preds[key] = tmp_df[TARGET_COL].values

inverse_scale_preds

{'test.csv': array([1.96888891, 2.64350311, 3.02531752, ..., 1.39617887, 3.23044205,
        4.07344255]),
 'train.csv': array([1.63601822, 1.70219333, 0.66430281, ..., 1.31385225, 1.13543401,
        2.24268852]),
 'val.csv': array([1.03696876, 0.89421862, 1.23962263, ..., 0.83946889, 1.18357954,
        0.94447362])}

In [37]:
# 標準化された値を復元した場合の評価スコア
for csv_name, y_true in zip([train_csv, val_csv, test_csv],
                            [train_df[TARGET_COL].values, val_df[TARGET_COL].values, test_df[TARGET_COL].values]):
  eval_scores[std_model_name][csv_name] = evaluete_sk_metrics(y_true=y_true,
                                                              y_pred=inverse_scale_preds[csv_name],
                                                              sklearn_metrics=SK_METRICS)

pprint.pprint(eval_scores)

{'ShallowNN': {'test.csv': {'explained_variance_score': 0.4349276989151253,
                            'mean_squared_error': 0.8056064330265855,
                            'r2_score': 0.4010434537196287},
               'train.csv': {'explained_variance_score': 0.4912248014795293,
                             'mean_squared_error': 0.7051086596845951,
                             'r2_score': 0.46263285548072286},
               'val.csv': {'explained_variance_score': 0.48019983175773084,
                           'mean_squared_error': 0.7613552447055286,
                           'r2_score': 0.45156132683995154}},
 'std_ShallowNN': {'std_test.csv': {'explained_variance_score': 0.746095717799165,
                                    'mean_squared_error': 0.26068363319909493,
                                    'r2_score': 0.7456855164473855},
                   'std_train.csv': {'explained_variance_score': 0.7600809596560849,
                                     'mean_squared_error': 